In [1]:
import numpy, pandas, sys, cv2, os
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.utils import plot_model
import keras, h5py
from Helpers.SteelDataGenerator import SteelDataGenerator as sd_gen

C:\Programs\Anaconda3\envs\ai\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
neu_data_dir = 'neu-data'
train_dir = os.path.join(neu_data_dir, 'train')
val_dir = os.path.join(neu_data_dir, 'val')
test_dir = os.path.join(neu_data_dir, 'test')
img_width, img_height = 200, 200
input_shape = (img_width, img_height, 3)
epochs = 1000
batch_size = 8
nb_train_samples = 1254
nb_validation_samples = 270    
nb_test_samples = 276
classes = 6
train_generator, val_generator, test_generator = sd_gen.get_data_generators(train_dir, val_dir, test_dir, img_width, img_height, batch_size)

Found 1254 images belonging to 6 classes.
Found 270 images belonging to 6 classes.
Found 276 images belonging to 6 classes.


In [12]:
check_point = keras.callbacks.ModelCheckpoint('LeNet4', monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [13]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6))
model.add(Activation('softmax'))



In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [15]:
from keras.callbacks import TensorBoard

In [16]:
tbCallBack = TensorBoard(log_dir='Graph',
                         histogram_freq=0,
                         write_graph=True,
                         write_images=True)

In [17]:
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=nb_validation_samples // batch_size,
        verbose=1, 
        callbacks=[check_point, early_stopping, tbCallBack])

Epoch 1/1000
156/156 [==============================] - 14s 88ms/step - loss: 1.7053 - acc: 0.2786 - val_loss: 1.7531 - val_acc: 0.2121

Epoch 00001: val_acc improved from -inf to 0.21212, saving model to LeNet4
Epoch 2/1000
156/156 [==============================] - 11s 70ms/step - loss: 1.5026 - acc: 0.3852 - val_loss: 1.9888 - val_acc: 0.2273

Epoch 00002: val_acc improved from 0.21212 to 0.22727, saving model to LeNet4
Epoch 3/1000
156/156 [==============================] - 11s 69ms/step - loss: 1.4707 - acc: 0.4009 - val_loss: 2.1504 - val_acc: 0.2121

Epoch 00003: val_acc did not improve
Epoch 4/1000
156/156 [==============================] - 11s 69ms/step - loss: 1.4073 - acc: 0.4669 - val_loss: 2.4256 - val_acc: 0.1856

Epoch 00004: val_acc did not improve
Epoch 5/1000
156/156 [==============================] - 11s 68ms/step - loss: 1.3239 - acc: 0.4971 - val_loss: 2.3218 - val_acc: 0.3485

Epoch 00005: val_acc improved from 0.22727 to 0.34848, saving model to LeNet4
Epoch 6/10


Epoch 00044: val_acc improved from 0.71212 to 0.71591, saving model to LeNet4
Epoch 45/1000
156/156 [==============================] - 10s 65ms/step - loss: 0.3313 - acc: 0.9076 - val_loss: 2.4551 - val_acc: 0.6932

Epoch 00045: val_acc did not improve
Epoch 46/1000
156/156 [==============================] - 10s 65ms/step - loss: 0.3875 - acc: 0.8838 - val_loss: 2.2274 - val_acc: 0.7121

Epoch 00046: val_acc did not improve
Epoch 47/1000
156/156 [==============================] - 10s 65ms/step - loss: 0.3940 - acc: 0.8806 - val_loss: 1.8096 - val_acc: 0.7689

Epoch 00047: val_acc improved from 0.71591 to 0.76894, saving model to LeNet4
Epoch 48/1000
156/156 [==============================] - 10s 65ms/step - loss: 0.3405 - acc: 0.8988 - val_loss: 2.0609 - val_acc: 0.6932

Epoch 00048: val_acc did not improve
Epoch 49/1000
156/156 [==============================] - 10s 65ms/step - loss: 0.3419 - acc: 0.8862 - val_loss: 2.1704 - val_acc: 0.7273

Epoch 00049: val_acc did not improve
Epoch

In [18]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Точность на тестовых данных: %.2f%%" % (scores[1]*100))

Точность на тестовых данных: 78.68%


In [19]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 198, 198, 32)      896       
_________________________________________________________________
activation_6 (Activation)    (None, 198, 198, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 97, 97, 32)        9248      
_________________________________________________________________
activation_7 (Activation)    (None, 97, 97, 32)        0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 46, 46, 64)        18496     
__________